In [6]:
"""
Multiply Manning's N by 2 in buffered terrace polygons
------------------------------------------------------
requirements:
    pip install xarray rioxarray geopandas rasterio numpy
"""

import xarray as xr
import rioxarray         # CRS helpers for xarray
import geopandas as gpd
from rasterio import features
import numpy as np
from pathlib import Path

# -----------------------------------------------------------------
# 1. file paths – EDIT to match your setup
# -----------------------------------------------------------------
STATICMAPS  = Path(r"C:\Users\jmsch\2025-06-02_wflow_qgis\wflow_reforest_and_terracing_all_subcatchments\staticmaps.nc")            # original NetCDF
GPKG        = Path(r"C:\Users\jmsch\2025-06-02_wflow_qgis\terraces_all_subcatchments.gpkg")            # GeoPackage with polygons
LAYER_NAME  = "terraces_all_subcatchments"                       # layer name inside the GPKG
OUTFILE     = Path(r"C:\Users\jmsch\2025-06-02_wflow_qgis\wflow_reforest_and_terracing_all_subcatchments\staticmaps_with_Nterrace.nc")
BUFFER_FACTOR = 0.5     # buffer = factor × cell‑width  (set 0 for no buffer)

# -----------------------------------------------------------------
# 2. load Manning's N layer
# -----------------------------------------------------------------
ds = xr.open_dataset(STATICMAPS)
if "N" not in ds:
    raise KeyError("Couldn't find variable 'N' in staticmaps.nc")
N = ds["N"]

# Attach CRS if missing
if not N.rio.crs:
    # >>> replace with your model CRS if necessary
    N = N.rio.write_crs("EPSG:4326")

transform   = N.rio.transform()
xres, yres  = abs(transform.a), abs(transform.e)   # pixel size
buf_dist    = BUFFER_FACTOR * max(xres, yres)     # one‑sided buffer

# -----------------------------------------------------------------
# 3. read & buffer terrace polygons
# -----------------------------------------------------------------
gdf = gpd.read_file(GPKG, layer=LAYER_NAME).to_crs(N.rio.crs)

if buf_dist > 0:
    gdf["geometry"] = gdf.geometry.buffer(buf_dist)  # buffer outward

# -----------------------------------------------------------------
# 4. rasterise – centroid test with buffered polygons
#    Rasterio uses “center‑of‑pixel” default (all_touched=False)
# -----------------------------------------------------------------
mask = features.rasterize(
    ((geom, 1) for geom in gdf.geometry if not geom.is_empty),
    out_shape   = N.shape[-2:],      # (rows, cols)
    transform   = transform,
    fill        = 0,
    dtype       = "uint8",
    all_touched = False              # centroid test
)

# -----------------------------------------------------------------
# 5. apply the multiplier only where mask == 1
# -----------------------------------------------------------------
N_new = N.where(mask == 0, N * 2)
surfstor = np.where(mask == 1, 0.05, 0)

# *** choose ONE of the two options below ***
# Option A – replace original N
ds["N"] = N_new
#ds["surfstor"] = surfstor
#ds["surfstor"] = (("latitude", "longitude"), surfstor)
#ds["surfstor"].attrs["units"] = "m"

# Option B – add a separate override layer
# ds["N_terrace"] = N_new

# -----------------------------------------------------------------
# 6. write updated NetCDF
# -----------------------------------------------------------------
ds.to_netcdf(OUTFILE)
print(f"Written: {OUTFILE.resolve()}")


C:\Users\jmsch\AppData\Local\Temp\ipykernel_16292\1353097761.py:47: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf["geometry"] = gdf.geometry.buffer(buf_dist)  # buffer outward


Written: C:\Users\jmsch\2025-06-02_wflow_qgis\wflow_reforest_and_terracing_all_subcatchments\staticmaps_with_Nterrace.nc


In [ ]:
# import xarray as xr
# import rioxarray
# import geopandas as gpd
# from rasterio import features
# import numpy as np
# from pathlib import Path

# # ------------------------------
# # File paths & settings – edit here
# # ------------------------------
# STATICMAPS  = Path(r"C:\Users\jmsch\2025-06-02_wflow_qgis\wflow_terracing_sc2\staticmaps_with_Nterrace.nc")
# GPKG        = Path(r"C:\Users\jmsch\2025-06-02_wflow_qgis\terraces_sc2.gpkg")
# LAYER_NAME  = "terraces_sc2"
# OUTFILE     = Path(r"C:\Users\jmsch\2025-06-02_wflow_qgis\wflow_terracing_sc2\staticmaps_with_Nterrace_and_surfstor.nc")
# BUFFER_FACTOR = 0.5  # buffer factor × pixel size

# # ------------------------------
# # Load dataset and terraces
# # ------------------------------
# ds = xr.open_dataset(STATICMAPS)
# if "N" not in ds:
#     raise KeyError("Variable 'N' not found in dataset (used to get CRS and shape)")
# N = ds["N"]

# # Attach CRS if missing (adjust CRS as needed)
# if not N.rio.crs:
#     N = N.rio.write_crs("EPSG:4326")

# transform = N.rio.transform()
# xres, yres = abs(transform.a), abs(transform.e)
# buf_dist = BUFFER_FACTOR * max(xres, yres)

# gdf = gpd.read_file(GPKG, layer=LAYER_NAME).to_crs(N.rio.crs)

# if buf_dist > 0:
#     gdf["geometry"] = gdf.geometry.buffer(buf_dist)

# # ------------------------------
# # Rasterize terraces (buffered)
# # ------------------------------
# mask = features.rasterize(
#     ((geom, 1) for geom in gdf.geometry if not geom.is_empty),
#     out_shape=N.shape[-2:],
#     transform=transform,
#     fill=0,
#     dtype="uint8",
#     all_touched=False
# )

# # ------------------------------
# # Create surfstor layer: 0.05 m inside terraces, 0 outside
# # ------------------------------
# surfstor = np.where(mask == 1, 0.05, 0.0)
# ds["surfstor"] = (("y", "x"), surfstor)
# ds["surfstor"].attrs["units"] = "m"

# # ------------------------------
# # Save updated NetCDF
# # ------------------------------
# ds.to_netcdf(OUTFILE)
# print(f"Surfstor layer added and saved to: {OUTFILE.resolve()}")


C:\Users\jmsch\AppData\Local\Temp\ipykernel_13948\1099166849.py:36: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf["geometry"] = gdf.geometry.buffer(buf_dist)


Surfstor layer added and saved to: C:\Users\jmsch\2025-06-02_wflow_qgis\wflow_terracing_sc2\staticmaps_with_Nterrace_and_surfstor.nc
